In [ ]:
import requests
import json
import time
import os
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# === CONFIGURATION ===
INPUT_FILE = "bloomberght_2011_2025_links_with_wayback.json"
OUTPUT_FILE = "bloomberght_articles_enriched.json"
TEMP_FILE = "bloomberght_articles_partial.json"
MAX_RETRIES = 2
SLEEP_BETWEEN_REQUESTS = 0.5
SAVE_EVERY = 1000
NUM_THREADS = 20

# === Load input file ===
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# === Resume if temp file exists ===
enriched_data = []
start_index = 0

if os.path.exists(TEMP_FILE):
    with open(TEMP_FILE, "r", encoding="utf-8") as f:
        enriched_data = json.load(f)
        start_index = len(enriched_data)
    print(f"🔁 Resuming from index {start_index}...")

# === User-Agent header ===
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; ArticleScraper/1.0)"
}

# === Scraping function ===
def scrape_article(entry):
    url = entry["wayback_raw_url"]
    for _ in range(MAX_RETRIES):
        try:
            response = requests.get(url, headers=headers, timeout=15)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")

                title = soup.select_one("h1")
                sub = soup.select_one(".amp-r-margin")
                time_tag = soup.select_one(".article-time")
                paragraphs = soup.find_all("p")

                return {
                    "title": title.get_text(strip=True) if title else "",
                    "subarticle": sub.get_text(strip=True) if sub else "",
                    "article_time": time_tag.get_text(strip=True) if time_tag else "",
                    "article_body": "\n".join(p.get_text(strip=True) for p in paragraphs)
                }
        except Exception:
            pass
        time.sleep(SLEEP_BETWEEN_REQUESTS)

    return {
        "title": "",
        "subarticle": "",
        "article_time": "",
        "article_body": ""
    }

# === Multithreaded execution ===
to_process = data[start_index:start_index+1000]
with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    futures = {executor.submit(scrape_article, entry): i for i, entry in enumerate(to_process)}
    temp_batch = []

    for count, future in enumerate(tqdm(as_completed(futures), total=len(futures), desc="Scraping")):
        index = futures[future]
        entry = to_process[index]
        result = future.result()
        full_entry = {**entry, **result}
        enriched_data.append(full_entry)
        temp_batch.append(full_entry)

        # Save every 1000 entries
        if len(enriched_data) % SAVE_EVERY == 0:
            with open(TEMP_FILE, "w", encoding="utf-8") as f:
                json.dump(enriched_data, f, ensure_ascii=False, indent=2)
            print(f"💾 Saved 1000 more entries (i.e. up to entry #{start_index + len(enriched_data)}) to the file...")

# === Final save ===
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(enriched_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Done! Final enriched dataset saved to: {OUTPUT_FILE}")


In [1]:
import requests
import json
import time
import os
from lxml import html
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# === CONFIGURATION ===
INPUT_FILE = "bloomberght_2011_2025_links_with_wayback.json"
OUTPUT_FILE = "bloomberght_articles_enriched_xpath.json"
TEMP_FILE = "bloomberght_articles_partial_xpath.json"
MAX_RETRIES = 2
SLEEP_BETWEEN_REQUESTS = 0.5
SAVE_EVERY = 1000
NUM_THREADS = 20

# === Load input file ===
with open(INPUT_FILE, "r", encoding="utf-8") as f:
    data = json.load(f)

# === Resume from checkpoint if exists ===
enriched_data = []
start_index = 0
if os.path.exists(TEMP_FILE):
    with open(TEMP_FILE, "r", encoding="utf-8") as f:
        enriched_data = json.load(f)
        start_index = len(enriched_data)
    print(f"🔁 Resuming from index {start_index}...")

# === Headers for requests ===
headers = {
    "User-Agent": "Mozilla/5.0 (compatible; ArticleScraper/1.0)"
}

# === Scraping Function (XPath using original_url) ===
def scrape_article(entry):
    url = entry["original_url"]
    for _ in range(MAX_RETRIES):
        try:
            response = requests.get(url, headers=headers, timeout=15)
            if response.status_code == 200:
                tree = html.fromstring(response.content)

                title = tree.xpath("/html/body/main/div[1]/div[2]/div[1]/article/div[1]/div[2]/h1/text()")
                summary = tree.xpath("/html/body/main/div[1]/div[2]/div[1]/article/div[1]/div[2]/ul/li/h2/text()")
                body = tree.xpath("/html/body/main/div[1]/div[2]/div[1]/article/div[2]//text()")
                date = tree.xpath("/html/body/main/div[1]/div[2]/div[1]/article/div[1]/div[4]/div/div[1]/div[1]/text()")

                return {
                    "title": title[0].strip() if title else "",
                    "subarticle": summary[0].strip() if summary else "",
                    "article_time": date[0].strip() if date else "",
                    "article_body": "\n".join([t.strip() for t in body if t.strip()])
                }
        except Exception:
            pass
        time.sleep(SLEEP_BETWEEN_REQUESTS)

    return {
        "title": "",
        "subarticle": "",
        "article_time": "",
        "article_body": ""
    }

# === Multithreaded Processing ===
to_process = data[start_index:]
with ThreadPoolExecutor(max_workers=NUM_THREADS) as executor:
    futures = {executor.submit(scrape_article, entry): i for i, entry in enumerate(to_process)}
    temp_batch = []

    for count, future in enumerate(tqdm(as_completed(futures), total=len(futures), desc="Scraping (XPath)")):
        index = futures[future]
        entry = to_process[index]
        result = future.result()
        enriched_entry = {**entry, **result}
        enriched_data.append(enriched_entry)
        temp_batch.append(enriched_entry)

        # Save every 1000
        if len(enriched_data) % SAVE_EVERY == 0:
            with open(TEMP_FILE, "w", encoding="utf-8") as f:
                json.dump(enriched_data, f, ensure_ascii=False, indent=2)
            print(f"💾 Saved 1000 more entries (up to #{start_index + len(enriched_data)}) to the file...")

# === Final Save ===
with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(enriched_data, f, ensure_ascii=False, indent=2)

print(f"\n✅ Done! Final enriched dataset saved to: {OUTPUT_FILE}")


🔁 Resuming from index 1000...


Scraping (XPath):   0%|                                                        | 1003/464680 [00:37<8:05:21, 15.92it/s]

💾 Saved 1000 more entries (up to #3000) to the file...


Scraping (XPath):   0%|▏                                                       | 2011/464680 [01:18<3:32:36, 36.27it/s]

💾 Saved 1000 more entries (up to #4000) to the file...


...

Scraping (XPath): 100%|█████████████████████████████████████████████████████▉| 463782/464680 [8:45:50<00:25, 35.28it/s]

💾 Saved 1000 more entries (up to #465000) to the file...


Scraping (XPath): 100%|██████████████████████████████████████████████████████| 464680/464680 [8:46:24<00:00, 14.71it/s]

💾 Saved 1000 more entries (up to #466000) to the file...



✅ Done! Final enriched dataset saved to: bloomberght_articles_enriched_xpath.json
